# Day 1

## Part 1
Count number of times depth reading differs from previous

In [1]:
import pandas as pd

d = pd.read_csv('data/aoc-1.1.txt', names=['depth'])
d.head()

,depth
0,198
1,208
2,209
3,212
4,213


In [2]:
d['prev'] = d.shift(1)
d['increase'] = d['depth'] > d['prev']
d.increase.sum()

1791

## Part 2
Now use the sum of a moving 3-measurement window

In [19]:
d = pd.read_csv('data/aoc-1.1.txt', names=['depth'])
d['prev'] = d.shift(1)
d['prev2'] = d['prev'].shift(1)
d['block'] = d.iloc[:, 0:3].sum(axis=1) # sum the first three columns
d.dropna(inplace=True) # remove rows without valid block scores
d['pblock'] = d['block'].shift(1)
d['block_inc'] = d['block'] > d['pblock']
d.block_inc.sum()

1822

# Day 2: Dive!

In [4]:
import pandas as pd

d = pd.read_csv('data/aoc-2.txt', names=['direction', 'distance'], delim_whitespace=True)
d.head()

,direction,distance
0,forward,9
1,forward,7
2,down,7
3,down,3
4,forward,2


In [9]:
d_horizontal = d['distance'][d['direction'] == 'forward'].sum()
d_up = d['distance'][d['direction'] == 'up'].sum()
d_down = d['distance'][d['direction'] == 'down'].sum()
d_vertical = d_down - d_up
d_horizontal * d_vertical

1882980

## Part 2
We actually need to track `aim`
* `down X` *increases* aim by `X`
* `up X` *decreases* aim by `X`
* `forward X` (1) increases horizontal positional by `X` and (2) increases depth by `aim` *multiplied* by `X`

In [13]:
aim = 0
dist = 0
depth = 0

for ii in d.index:
    direction = d['direction'][ii]
    X = d['distance'][ii]
    if direction == 'forward':
        dist += X
        depth += aim * X
    elif direction == 'up':
        aim -= X
    else:
        aim += X
dist * depth

1971232560

# Day 3: Binary Diagnostics

The report is rows of binary values. The most common value in each column is the `gamma` and the least common value in each column is the `epsilon`. Multiplying these numbers returns the power consumption.

In [82]:
import numpy as np
import pandas as pd

d = pd.read_csv('data/aoc-3.txt', names=['report'], dtype='str')
d.head()

,report
0,011110011110
1,101101001111
2,000000010101
3,100111001010
4,110000011010


In [83]:
ds = d['report'].apply(lambda x: pd.Series(list(x)))
ds = ds.astype(int)
ds.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,0,1,1,1,1,0,0,1,1,1,1,0
1,1,0,1,1,0,1,0,0,1,1,1,1
2,0,0,0,0,0,0,0,1,0,1,0,1
3,1,0,0,1,1,1,0,0,1,0,1,0
4,1,1,0,0,0,0,0,1,1,0,1,0


In [84]:
gamma_digits

[0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1]

In [85]:
nrow, ncol = ds.shape
gamma_digits = []
epsilon_digits = []
for col in range(ncol):
    ones = ds[col].sum()
    if ones > 500:
        gamma_digits.append(1)
        epsilon_digits.append(0)
    else:
        gamma_digits.append(0)
        epsilon_digits.append(1)

In [86]:
# convert binary to decimal
gamma = 0
epsilon = 0
for ii in range(ncol):
    gamma += gamma_digits[ncol-ii-1] * 2**(ii)
    epsilon += epsilon_digits[ncol-ii-1] * 2**(ii)

In [87]:
gamma*epsilon

775304

## Part 2
Now verify `life support rating` which is `oxygen generater rating` * `CO2 scrubber rating`

**oxygen generator rating**
* determine most common value in current bit position and keep only number with that bit in that position. If `0` and `1` are equally common, keep `1`.

**CO2 scrubber rating**
* determine the least common value. If `0` and `1` are equally common, keep `0`.

After determining *bit criteria*, discard numbers which don't match the bit criteria. If you only have one number left, stop, this is the rating. Otherwise, repeat, considering next bit to the right.

In [111]:
def most_common(ser, target='o'):
    n_length = ser.shape[0]
    if ser.sum() > n_length/2:
        return(1)
    elif ser.sum() < n_length/2:
        return(0)
    if ser.sum() == n_length/2:
        if target == 'o':
            return(1)
        else:
            return(0)

ds_all = ds.copy()

oxy_val = 0
col_id = 0
rows_left = ds_all.shape[0]
ds_oxy = ds_all.copy()
while col_id < ncol and rows_left > 0:
    print(col_id)
    bit_val = most_common(ds_oxy[col_id])
    ds_oxy = ds_oxy[ds_oxy[col_id] == bit_val]
    rows_left = ds_oxy.shape[0]
    col_id += 1

0
1
2
3
4
5
6
7
8
9
10
11


In [112]:
ds_oxy

,0,1,2,3,4,5,6,7,8,9,10,11
844,0,0,0,1,1,1,1,1,1,1,0,1


In [119]:
def least_common(ser, target='o'):
    n_length = ser.shape[0]
    if ser.sum() < n_length/2:
        return(1)
    elif ser.sum() > n_length/2:
        return(0)
    if ser.sum() == n_length/2:
        if target == 'o':
            return(0)
        else:
            return(1)
        
co_val = 0
col_id = 0
rows_left = ds_all.shape[0]
ds_co = ds_all.copy()
while col_id < ncol and rows_left > 1:
    bit_val = least_common(ds_co[col_id])
    ds_co = ds_co[ds_co[col_id] == bit_val]
    rows_left = ds_co.shape[0]
    col_id += 1

In [121]:
ds_co

,0,1,2,3,4,5,6,7,8,9,10,11
639,1,0,1,0,1,0,0,0,0,1,0,1


In [122]:
oxy = 0
co = 0
for ii in range(ncol):
    oxy += ds_oxy.iloc[0,ncol-ii-1] * 2**(ii)
    co += ds_co.iloc[0, ncol-ii-1] * 2**(ii)

In [123]:
oxy*co

1370737

# Day 7: The Treachery of Whales

A whale is coming to eat the submarine. We need to get all of the crab submarines to the same position so they can blast a hole the ocean floor and grant us access to the underground tunnels in which to hide. But, we need the most fuel-efficient position to align on.

In [131]:
d = pd.read_csv('data/aoc-7.txt', header=None)
d.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1101,1,29,67,1102,0,1,65,1008,65,...,1071,538,1242,1247,116,117,1638,675,498,570


In [141]:
d_val = np.array(d.iloc[0])
d_val.shape

(1000,)

In [196]:
# brute force
def check_dists(d, val):
    dd = d - val
    return(np.abs(dd).sum())

n_crab = d_val.shape[0]
test_range = d_val.max() - d_val.min()

tests = np.zeros(test_range)
for ii in range(d_val.min(), d_val.max()):
    tests[ii] = check_dists(d_val, ii)
tests.min()

355764.0

In [217]:
# Also, you can always just start at the median
check_dists(d_val, np.percentile(d_val, 50))

355764.0

## Part 2
Whoops, looks like the crabs actually need to use more fuel for each move. The first move costs 1 fuel, the second costs 2, etc.

In [203]:
def check_dist2(d, val):
    d1 = np.abs(d - val)
    d_adj = (d1*(d1+1))/2
    return(d_adj.sum())

tests = np.zeros(test_range)
for ii in range(d_val.min(), d_val.max()):
    tests[ii] = check_dist2(d_val, ii)
tests.min()

99634572.0